In [61]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
from ipywidgets import Dropdown, interactive
import geopandas as gpd
import pandas as pd
import json
from pathlib import Path
from IPython.display import display

In [2]:
DATOS_PROCESADOS = Path("../../datos/procesados/")

In [3]:
data = pd.read_csv(DATOS_PROCESADOS / "agebs.csv")

In [4]:
shapes = DATOS_PROCESADOS.glob("*.shp")

In [5]:
geodatos = pd.concat((gpd.read_file(file) for file in shapes), ignore_index = True)

In [6]:
geodatos.set_index("CVEGEO", inplace = True)
data.set_index("CVEGEO", inplace = True)

In [7]:
datos = geodatos.join(data, how = 'right',)
datos.dropna(inplace = True)

In [166]:
datos = datos.to_crs(epsg = 4326)

In [267]:
def geo_center(df):
    _bounds = df.bounds
    xmin, xmax, ymin, ymax = _bounds.minx.min(), _bounds.maxx.max(), _bounds.miny.min(), _bounds.maxy.max()
    y_center = (ymax + ymin) / 2
    x_center = (xmax + xmin) / 2
    center = (y_center, x_center)
    
    return center

In [268]:
_orden = ['(70, 100]', '(50, 70]', '(34, 50]', '(18, 34]', '[ 0, 18]', 'Una vivienda particular habitada', 'Sin viviendas particulares habitadas', 'No disponible']
_colores = ["#dc0d12","#e9686b","#fbe1e1","#dff4f9","#81d1e6","#dc0d7a","#e46800","#a20c4b"]

In [269]:
_escala = dict(zip(_orden, _colores))

In [270]:
def filtra_datos(estado, municipio):
    df = datos.copy()
    mask_estado = df['Entidad Federativa'] == estado
    
    # agrega - para todos
    if municipio == "---":
        working_df = df[mask_estado].copy()
    else:
        mask_municipio = df['Municipio'] == municipio
        working_df = df[mask_estado & mask_municipio].copy()
    
    viz = crea_visualizacion(working_df)
    viz
    return display(viz)

In [271]:
def crea_visualizacion(datos):
    _centro = geo_center(datos)
    
    m = Map(center = _centro, zoom = 10, basemap = basemaps.OpenStreetMap.BlackAndWhite, scroll_wheel_zoom = True)
    
    for option in datos['Rango de pobreza (%)'].unique():
        
        option_data = datos[datos['Rango de pobreza (%)'] == option]
        
        geo_data = GeoData(
            geo_dataframe = option_data,
            style={'color': _escala[option], 'fillColor': _escala[option], 'opacity':0.35, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.7},  
            hover_style={'fillColor': _escala[option] , 'fillOpacity': 0.9},
            name = f'{option}'
        )
        
        # add layers
        m.add_layer(geo_data)
    
    m.add_control(LayersControl())
        
    return m

In [272]:
estado_widget = Dropdown(options = list(data['Entidad Federativa'].unique()), description = 'Estado')
municipio_widget = Dropdown(options = ["---"] + list(data[data['Entidad Federativa']== estado_widget.value]['Municipio'].unique()), description = 'Muncipio')

In [273]:
def on_state_changed(change):
    municipio_widget.options = ["---"] + list(data[data['Entidad Federativa'] == estado_widget.value]['Municipio'].unique())

In [274]:
estado_widget.observe(on_state_changed, names = 'value')

# Pobreza Urbana en México

Inspirado por los mapas de [@moaimx](https://twitter.com/moaimx) ([aquí el hilo de twitter](https://twitter.com/moaimx/status/1141534141487468549)) y la más reciente adición al ecosistema Jupyter: [Voilà](https://github.com/QuantStack/voila) por [@QuantStack](https://twitter.com/QuantStack) decidí intentar hacer algo juntando ambos.

Gracias especiales a [@Coneval](https://twitter.com/Coneval) por los datos.

_aclaración_: Esta aplicación web no esta agregando ningún análisis a los datos y en ciertos municipios puede que no se vea tanto detalle por la naturaleza de su geografía (miren Ensenada en Baja California, por ejemplo).

_aclaración 2_: Se tarda un poco en cargar el mapa así que tengan paciencia porfa

In [275]:
interactive(filtra_datos, estado = estado_widget, municipio = municipio_widget)

interactive(children=(Dropdown(description='Estado', options=('Aguascalientes', 'Baja California', 'Baja Calif…